<a href="https://colab.research.google.com/github/luismiguelmartinluengo/Test_in_Colab/blob/main/Introduccion_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col, count, mean, avg
from pyspark.sql import types

In [2]:
ruta_dataset_1 = 'sample_data/csv1.txt'
ruta_dataset_2 = 'sample_data/csv2.txt'

In [3]:
spark_session = (SparkSession.builder.appName("SparkSQLExampleApp").getOrCreate())

In [4]:
spark_session.version

'3.5.4'

In [5]:
import os
for item in os.listdir('sample_data/'):
  print(item)

README.md
anscombe.json
csv2.txt
csv1.txt
mnist_train_small.csv
mnist_test.csv
california_housing_test.csv
california_housing_train.csv


In [6]:
df1 = spark_session.read.csv(ruta_dataset_1, header = True, sep = ';')

In [8]:
df1.show()

+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  3| 23|  tal|4.3|
|  4| 24|estas|4.4|
|  5| 25|   tu|4.5|
+---+---+-----+---+



In [9]:
df1.describe().show()

+-------+------------------+------------------+-----+-------------------+
|summary|                 a|                 b|    c|                  d|
+-------+------------------+------------------+-----+-------------------+
|  count|                 5|                 5|    5|                  5|
|   mean|               3.0|              23.0| NULL|                4.3|
| stddev|1.5811388300841898|1.5811388300841898| NULL|0.15811388300841897|
|    min|                 1|                21|estas|                4.1|
|    max|                 5|                25|   tu|                4.5|
+-------+------------------+------------------+-----+-------------------+



In [10]:
df1.printSchema()

root
 |-- a: string (nullable = true)
 |-- b: string (nullable = true)
 |-- c: string (nullable = true)
 |-- d: string (nullable = true)



In [11]:
df1.isEmpty()

False

In [12]:
df1.columns

['a', 'b', 'c', 'd']

In [13]:
df1.count()

5

In [14]:
df1.head(2)

[Row(a='1', b='21', c='hola', d='4.1'), Row(a='2', b='22', c='que', d='4.2')]

In [15]:
df2 = spark_session.read.csv(ruta_dataset_2, header = True, sep = ';')
df2.show(2)

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  2|  2|  3|  4|
+---+---+---+---+
only showing top 2 rows



In [16]:
dfea = df1.exceptAll(df2)
dfea.show()

+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  3| 23|  tal|4.3|
|  5| 25|   tu|4.5|
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  4| 24|estas|4.4|
+---+---+-----+---+



In [17]:
dfea.describe().show()

+-------+------------------+------------------+-----+-------------------+
|summary|                 a|                 b|    c|                  d|
+-------+------------------+------------------+-----+-------------------+
|  count|                 5|                 5|    5|                  5|
|   mean|               3.0|              23.0| NULL|                4.3|
| stddev|1.5811388300841898|1.5811388300841898| NULL|0.15811388300841916|
|    min|                 1|                21|estas|                4.1|
|    max|                 5|                25|   tu|                4.5|
+-------+------------------+------------------+-----+-------------------+



In [18]:
df1.select('a','c').limit(3).show()

+---+----+
|  a|   c|
+---+----+
|  1|hola|
|  2| que|
|  3| tal|
+---+----+



In [19]:
df1.take(2)

[Row(a='1', b='21', c='hola', d='4.1'), Row(a='2', b='22', c='que', d='4.2')]

In [20]:
df1.head(2)

[Row(a='1', b='21', c='hola', d='4.1'), Row(a='2', b='22', c='que', d='4.2')]

In [21]:
filas = df1.limit(2).collect()
print(filas[0].asDict())
print(filas[1].asDict())

{'a': '1', 'b': '21', 'c': 'hola', 'd': '4.1'}
{'a': '2', 'b': '22', 'c': 'que', 'd': '4.2'}


In [23]:
union = df1.union(df2)
union.show()

+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  3| 23|  tal|4.3|
|  4| 24|estas|4.4|
|  5| 25|   tu|4.5|
|  1|  2|    3|  4|
|  2|  2|    3|  4|
|  2|  2|    3|  4|
|  2|  2|    3|  4|
|  2|  2|    3|  4|
+---+---+-----+---+



In [24]:
dfwc = df1.withColumns({'nueva_columna':when(df1.a == '1', 'cazado').otherwise('no cazado'),
                        'nueva_columna2':lit(6)})
dfwc.show()

+---+---+-----+---+-------------+--------------+
|  a|  b|    c|  d|nueva_columna|nueva_columna2|
+---+---+-----+---+-------------+--------------+
|  1| 21| hola|4.1|       cazado|             6|
|  2| 22|  que|4.2|    no cazado|             6|
|  3| 23|  tal|4.3|    no cazado|             6|
|  4| 24|estas|4.4|    no cazado|             6|
|  5| 25|   tu|4.5|    no cazado|             6|
+---+---+-----+---+-------------+--------------+



In [25]:
dfwc.withColumnRenamed('nueva_columna', 'es_cazado').drop('nueva_columna2').show()

+---+---+-----+---+---------+
|  a|  b|    c|  d|es_cazado|
+---+---+-----+---+---------+
|  1| 21| hola|4.1|   cazado|
|  2| 22|  que|4.2|no cazado|
|  3| 23|  tal|4.3|no cazado|
|  4| 24|estas|4.4|no cazado|
|  5| 25|   tu|4.5|no cazado|
+---+---+-----+---+---------+



In [30]:
union = union.dropDuplicates()#sin parámetros hace lo mismo que distinct
union = union.distinct()
union.show()

+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  3| 23|  tal|4.3|
|  5| 25|   tu|4.5|
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  4| 24|estas|4.4|
|  1|  2|    3|  4|
|  2|  2|    3|  4|
+---+---+-----+---+



In [32]:
datos_nuevo_df = [('n11','n12','n13','n14'),
                  ('n21','n22','n23','n24')]
columnas_nuevo_df = union.columns
nuevo_df = spark_session.createDataFrame(data = datos_nuevo_df, schema = columnas_nuevo_df)
nuevo_df.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|n11|n12|n13|n14|
|n21|n22|n23|n24|
+---+---+---+---+



In [33]:
union = union.union(nuevo_df)
union.show()

+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  3| 23|  tal|4.3|
|  5| 25|   tu|4.5|
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  4| 24|estas|4.4|
|  1|  2|    3|  4|
|  2|  2|    3|  4|
|n11|n12|  n13|n14|
|n21|n22|  n23|n24|
+---+---+-----+---+



In [34]:
union.createOrReplaceTempView('union')
df_select = spark_session.sql('select * from union where a like "%1%"')
df_select.show()

+---+---+----+---+
|  a|  b|   c|  d|
+---+---+----+---+
|  1| 21|hola|4.1|
|  1|  2|   3|  4|
|n11|n12| n13|n14|
|n21|n22| n23|n24|
+---+---+----+---+



In [37]:
#Otra forma de filtro (como pandas)
df_filtrado = union.filter(col("a").contains("1"))
df_filtrado.show()

+---+---+----+---+
|  a|  b|   c|  d|
+---+---+----+---+
|  1| 21|hola|4.1|
|  1|  2|   3|  4|
|n11|n12| n13|n14|
|n21|n22| n23|n24|
+---+---+----+---+



In [40]:
print(df_filtrado.collect())

[Row(a='1', b='21', c='hola', d='4.1'), Row(a='1', b='2', c='3', d='4'), Row(a='n11', b='n12', c='n13', d='n14'), Row(a='n21', b='n22', c='n23', d='n24')]


In [41]:
df1.show()

+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  3| 23|  tal|4.3|
|  4| 24|estas|4.4|
|  5| 25|   tu|4.5|
+---+---+-----+---+



In [46]:
df1.a = df1.a.astype('int')
df1.b = df1.b.astype('int')
df1.d = df1.d.astype('float')
print(df1.dtypes)
df1.show()

[('a', 'string'), ('b', 'string'), ('c', 'string'), ('d', 'string')]
+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  3| 23|  tal|4.3|
|  4| 24|estas|4.4|
|  5| 25|   tu|4.5|
+---+---+-----+---+



In [52]:
dfnum = df1.withColumns({"a": df1.a.cast(types.IntegerType()),
                         "b": df1.b.cast(types.IntegerType()),
                         "d": df1.d.cast(types.FloatType())})
print(dfnum.dtypes)
dfnum.show()

[('a', 'int'), ('b', 'int'), ('c', 'string'), ('d', 'float')]
+---+---+-----+---+
|  a|  b|    c|  d|
+---+---+-----+---+
|  1| 21| hola|4.1|
|  2| 22|  que|4.2|
|  3| 23|  tal|4.3|
|  4| 24|estas|4.4|
|  5| 25|   tu|4.5|
+---+---+-----+---+



In [55]:
dfag = dfnum.groupBy('c').agg({'a':'sum', 'b':'mean', 'd':'max'})
dfag.show()

+-----+------+------+------+
|    c|sum(a)|avg(b)|max(d)|
+-----+------+------+------+
|   tu|     5|  25.0|   4.5|
|estas|     4|  24.0|   4.4|
|  que|     2|  22.0|   4.2|
| hola|     1|  21.0|   4.1|
|  tal|     3|  23.0|   4.3|
+-----+------+------+------+



In [73]:
dfag2 = dfnum.groupBy('c').agg(count("a").alias("suma_de_a"),
                               mean("b").alias("mediana de b"),
                               avg("d").alias("media de d"))
dfag2.show()

+-----+---------+------------+-----------------+
|    c|suma_de_a|mediana de b|       media de d|
+-----+---------+------------+-----------------+
|   tu|        1|        25.0|              4.5|
|estas|        1|        24.0|4.400000095367432|
|  que|        1|        22.0|4.199999809265137|
| hola|        1|        21.0|4.099999904632568|
|  tal|        1|        23.0|4.300000190734863|
+-----+---------+------------+-----------------+

